<a href="https://colab.research.google.com/github/Rubi24-S/rubishabong/blob/main/plantdiseaseproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # Mount Google Drive for dataset storage

!pip install tensorflow matplotlib seaborn scikit-learn  # Install required libraries



Mounted at /content/drive


In [ ]:
import zipfile
import os

dataset_path = "/content/drive/My Drive/archive.zip"  # Change this path as needed
extract_path = "/content/archive"

with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully!")


Dataset extracted successfully!


In [ ]:
import os
for category in os.listdir(extract_path + "/PlantVillage"):
    print(category)


Pepper__bell___healthy
Tomato__Target_Spot
Tomato_Early_blight
Potato___healthy
Tomato_Spider_mites_Two_spotted_spider_mite
Tomato_healthy
Tomato_Leaf_Mold
Potato___Early_blight
Tomato_Septoria_leaf_spot
Tomato__Tomato_mosaic_virus
Tomato_Bacterial_spot
Pepper__bell___Bacterial_spot
Tomato_Late_blight
Potato___Late_blight
Tomato__Tomato_YellowLeaf__Curl_Virus


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

# Define dataset directory
dataset_dir = "/content/archive"

# Define ImageDataGenerator with data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=30, zoom_range=0.2,
                                   horizontal_flip=True, validation_split=0.2)

# Load training and validation data
train_generator = train_datagen.flow_from_directory(dataset_dir, target_size=(128, 128),
                                                    batch_size=32, class_mode='categorical', subset='training')

val_generator = train_datagen.flow_from_directory(dataset_dir, target_size=(128, 128),
                                                  batch_size=32, class_mode='categorical', subset='validation')

print("Total classes:", len(train_generator.class_indices))


Found 33022 images belonging to 2 classes.
Found 8254 images belonging to 2 classes.
Total classes: 2


In [ ]:
# Import necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of Model
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,304,898 (12.61 MB)

 Trainable params: 3,304,898 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the Model
history = model.fit(train_generator, validation_data=val_generator, epochs=10, verbose=1)



/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 1236s 1s/step - accuracy: 0.4970 - loss: 0.7128 - val_accuracy: 0.5000 - val_loss: 0.6934
Epoch 2/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 1185s 1s/step - accuracy: 0.5025 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 3/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 1190s 1s/step - accuracy: 0.5033 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 4/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 1157s 1s/step - accuracy: 0.5012 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 5/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 1141s 1s/step - accuracy: 0.4971 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 6/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 1194s 1s/step - accuracy: 0.4909 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 7/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 1144s 1s/step - accuracy: 0.4990 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 8/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 1160s 1s/step - accuracy: 0.4945 -